In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import sys

import os
import numpy as np
import matplotlib.pyplot as plt
from nltk.tokenize import sent_tokenize, word_tokenize 
import warnings

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [86]:
from itertools import islice

data = {'Y': [], 'predict': []}
with open('drive/My Drive/xsy_test_result.txt', 'r') as infile:
  line_num = 0
  for line in infile:
    line = line.strip('\n ')
    if line_num % 3 == 0:
      data['Y'].append(line)
    elif line_num % 3 == 1:
      data['predict'].append(line)
    line_num += 1

print(data['Y'][999])
print(data['predict'][999])

so far only their scouts . but we have had reports of a small impi farther north over there .
i m not sure .


In [4]:
from nltk.tokenize import sent_tokenize, word_tokenize 
import warnings 
  
warnings.filterwarnings(action = 'ignore') 
  
import gensim 
from gensim.models import Word2Vec
import gensim.downloader

In [5]:
glove_vectors = gensim.downloader.load('glove-twitter-25')

[==================================================] 100.0% 104.8/104.8MB downloaded


In [76]:
# evaluation matirc
def greedy_matching(s1, s2):
  score = 0
  simi_sum = 0
  for w in s1:
    max_simi = max([glove_vectors.similarity(w, wi) for wi in s2])
    simi_sum += max_simi
  score += (simi_sum / len(s1))
  simi_sum = 0
  for w in s2:
    max_simi = max([glove_vectors.similarity(w, wi) for wi in s1])
    simi_sum += max_simi
  score += (simi_sum / len(s2))
  return score / 2

def embedding_average(s1, s2):
  e1_avg = np.average([glove_vectors[w] for w in s1], axis=0)
  e2_avg = np.average([glove_vectors[w] for w in s2], axis=0)
  score = glove_vectors.cosine_similarities(e1_avg, [e2_avg])
  return score[0]

def vector_extrema(s1, s2):
  def absmax(a):
    a_min = np.min(a, axis=0)
    a_max = np.max(a, axis=0)
    a_absmax = [max(a_min[i], a_max[i], key=abs) for i in range(len(a_min))]
    return a_absmax
  e1 = [glove_vectors[w] for w in s1]
  e2 = [glove_vectors[w] for w in s2]
  score = glove_vectors.cosine_similarities(absmax(e1), [absmax(e2)])
  return score[0]

In [87]:
data['Y'] = [s.split(' ') for s in data['Y']]
data['predict'] = [s.split(' ') for s in data['predict']]

clean_Y = [[w for w in w_arr if w in glove_vectors] for w_arr in data['Y']]
clean_predict = [[w for w in w_arr if w in glove_vectors] for w_arr in data['predict']]

In [78]:
print(greedy_matching(data['Y'][11], data['predict'][11]))
print(embedding_average(data['Y'][11], data['predict'][11]))
print(vector_extrema(data['Y'][11], data['predict'][11]))

0.9099748566364632
0.9541596
0.7761334


In [88]:
g_m = 0
e_a = 0
v_e = 0
all = 0
for y, p in zip(clean_Y, clean_predict):
  if len(y) != 0 and len(p) != 0:
    g_m += greedy_matching(y, p)
    e_a += embedding_average(y, p)
    v_e += vector_extrema(y, p)
    all += 1
g_m /= all
e_a /= all
v_e /= all

In [82]:
print("Transformer:")
print(g_m)
print(e_a)
print(v_e)
print(all)

Transformer:
0.8701796827196008
0.922560159541502
0.7281755257068748
996


In [89]:
print("seq2seq:")
print(g_m)
print(e_a)
print(v_e)
print(all)

seq2seq:
0.8706608828072472
0.9288320587873459
0.7190872244089842
1000


Codes below are useless, just for test.

In [8]:
glove_vectors.similarity('guy', 'man')

0.8295315

In [20]:
np.shape(glove_vectors['computer'])

(25,)

In [15]:
import numpy as np
a = [[1,2],[3,4],[5,6]]
np.average(a, axis=0)

array([3., 4.])

In [23]:
te1 = glove_vectors['man']
te2 = glove_vectors['guy']
s = glove_vectors.cosine_similarities(te1, [te2])
s[0]

0.8295315

In [39]:
a = [[1, 2, 5], [4, -5, 2]]
a_min = np.min(a, axis=0)
a_max = np.max(a, axis=0)
absmax = [max(a_min[i], a_max[i], key=abs) for i in range(len(a_min))]
print(absmax)

[4, -5, 5]


In [57]:
print(data['Y'][11])
print(data['predict'][11])

['i', 'dont', 'want', 'to', 'take', 'em', 'out', 'just', 'want', 'to', 'keep', 'em', 'interested']
['i', 'dont', 'know', 'what', 'you', 'mean']


In [52]:
a = " asd fdf df\n "
a.strip('\n ')

'asd fdf df'

In [85]:
if '' in glove_vectors:
  print('yes')
else:
  print('no')

yes


In [ ]:
import re

def normalizeString(s):
    s = s.lower()
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

